In [2]:
class Parent:
    pass

type(Parent)

type

In [3]:
type(int), type(list), type(object)

(type, type, type)

In [4]:
type(type)

type

Superclass of `metaclass` is `type`

`__init__` vs `__new__`

`__new__()` is intended mainly to allow subclasses of immutable types (like int, str, or tuple) to customize instance creation. It is also commonly overridden in custom metaclasses in order to customize class creation.

Because `__new__()` and `__init__()` work together in constructing objects (`__new__()` to create it, and `__init__()` to customize it), no non-None value may be returned by __init__(); doing so will cause a TypeError to be raised at runtime.

In [9]:
class MetaClassOne(type):
    def __new__(cls, *args):
        print(cls)
        print(args)

        # this line ensures the cls __init__ is called
        return type.__new__(cls, *args)

In [10]:
class ExampleClass(metaclass=MetaClassOne):
    int1 = 123

    def test():
        print('test')

<class '__main__.MetaClassOne'>
('ExampleClass', (), {'__module__': '__main__', '__qualname__': 'ExampleClass', 'int1': 123, 'test': <function ExampleClass.test at 0x7f60be27e310>})


An object’s type determines the operations that the object supports (e.g., “does it have a length?”) and also defines the possible values for objects of that type



# Overloading

Classes allow you to make instances of the class (i.e. new objects), but classes are objects

What is `type` of class?
 Class definition is a syntactic way of creating a new type

Metclasses help customise what happens in the `prepare` method or the `__init__`/`__new__` methods.
A.f()

Metaclasses are inheritable

In [15]:
class A:
    pass

type(A)

type

In [16]:
B = type('B', (), {}) # no difference between type and class
print(f'{B=}')

B=<class '__main__.B'>


In [26]:
# either
class A:
    pass

# OR 
def make_A():
    name = 'A'
    bases = ()

    a = 1
    b = 'hello'

    def f(self):
        return 117

    namespace = type.__prepare__(name, bases)
    body = (
'''
a = 1
b = 'hello'

def f(self,):
    return 117
'''
    )
    exec(body, globals(), namespace)

    A = type(name, bases, namespace)
    return A

In [28]:
A = make_A()

In [29]:
a = A()

In [30]:
a.f()

117

When class created, the various functions are defined and stored in some diciontary. Can edit this dictionary by overriding default `__prepare__` method

In [7]:
# creating a class is calling type()
# type(name, bases, namespace)
class C:
    pass

type('A', (), {})

TypeError: type.__new__() argument 2 must be tuple, not type

When a class definition is executed, the following steps occur:
- MRO entries are resolved;
- the appropriate metaclass is determined;
- the class namespace is prepared;
- the class body is executed;
- the class object is created.

Namespaces in Python. A namespace is a collection of currently defined symbolic names along with information about the object that each name references. You can think of a namespace as a dictionary in which the keys are the object names and the values are the objects themselves

In [9]:
class A:
    pass
A.__mro__

(__main__.A, object)

In [10]:
help(exec)

Help on built-in function exec in module builtins:

exec(source, globals=None, locals=None, /)
    Execute the given source in the context of globals and locals.
    
    The source may be a string representing one or more Python statements
    or a code object as returned by compile().
    The globals must be a dictionary and locals can be any mapping,
    defaulting to the current globals and locals.
    If only globals is given, locals defaults to it.



In [12]:
a = A()
isinstance(a, A), isinstance(A, type)

(True, True)

- `Metaclass.__prepare__` just returns the namespace object (a dictionary-like object as explained before).
- `Metaclass.__new__` returns the Class object.
- `Metaclass.__call__` returns whatever Metaclass.__new__ returned (and if it returned an instance of Metaclass it will also call Metaclass.__init__ on it).

In [16]:
import pandas as pd 
type.__prepare__('A', ())

{}

In [17]:
class Base:
    __slots__ = 'foo', 'bar'

In [20]:
help(type.__call__)

Help on wrapper_descriptor:

__call__(self, /, *args, **kwargs)
    Call self as a function.



In [21]:
# metaclass
class Meta(type):
    def __new__(cls, classname, bases, attributes):
        print(cls)

        return type.__new__(cls, classname, bases, attributes)

class A(metaclass=Meta):
    pass

<class '__main__.Meta'>


In [22]:
a = A()

NameError: name '__file__' is not defined

In [26]:
# not meta
from tracemalloc import start, take_snapshot

start()
before = take_snapshot()
a = A()
after = take_snapshot()

for stat in (stat for stat in after.compare_to(before, 'lineno') if stat.traceback[0].filename == 'metaclasses.ipynb'):
    print(stat)

# Example 

From [James Powell: Advanced Metaphors in Coding with Python](https://www.youtube.com/watch?v=R2ipPgrWypI&t=446s)

In [28]:
class Base:
    def bar(self, ):
        return 'bar'

assert hasattr(Base, 'bar')
# OR using a unit test that
def test_Base():
    # this fulfills all implicit assumptions about my code if it passes
    b = Base()
    b.bar

In [29]:
# python has "hooks" due to its rich runtime
def f():
    class Foo: # this is executable code
        pass
from dis import dis

dis(f)

  2           0 LOAD_BUILD_CLASS
              2 LOAD_CONST               1 (<code object Foo at 0x7f3486598ea0, file "/tmp/ipykernel_510800/2346271759.py", line 2>)
              4 LOAD_CONST               2 ('Foo')
              6 MAKE_FUNCTION            0
              8 LOAD_CONST               2 ('Foo')
             10 CALL_FUNCTION            2
             12 STORE_FAST               0 (Foo)
             14 LOAD_CONST               0 (None)
             16 RETURN_VALUE

Disassembly of <code object Foo at 0x7f3486598ea0, file "/tmp/ipykernel_510800/2346271759.py", line 2>:
  2           0 LOAD_NAME                0 (__name__)
              2 STORE_NAME               1 (__module__)
              4 LOAD_CONST               0 ('f.<locals>.Foo')
              6 STORE_NAME               2 (__qualname__)

  3           8 LOAD_CONST               1 (None)
             10 RETURN_VALUE


In [32]:
import builtins
# __build_class__ is called for building classes
help(__build_class__) # (cls, name, bases)

Help on built-in function __build_class__ in module builtins:

__build_class__(...)
    __build_class__(func, name, /, *bases, [metaclass], **kwds) -> class
    
    Internal helper function used by the class statement.



Look into `__init_subclass__` as well

In [37]:
# metaclasses construct the new classes
# __new__ creates a new CLASS, NOT AN INSTANCE

# class BaseMeta(type):
#     def __new__(cls, name, bases, body):
#         # example
#         if 'baz' not in body:
#             raise TypeError()
#         return super().__new__(cls, name, bases, body)

# class Base(metaclass=BaseMeta):
#     def bar(self,):
#         return self.baz()
from library import Base

class Derived(Base):
    def baz(self, ):
        return 'baz'

TypeError: 

In [1]:
from collections import namedtuple

help(namedtuple)

Help on function namedtuple in module collections:

namedtuple(typename, field_names, *, rename=False, defaults=None, module=None)
    Returns a new subclass of tuple with named fields.
    
    >>> Point = namedtuple('Point', ['x', 'y'])
    >>> Point.__doc__                   # docstring for the new class
    'Point(x, y)'
    >>> p = Point(11, y=22)             # instantiate with positional args or keywords
    >>> p[0] + p[1]                     # indexable like a plain tuple
    33
    >>> x, y = p                        # unpack like a regular tuple
    >>> x, y
    (11, 22)
    >>> p.x + p.y                       # fields also accessible by name
    33
    >>> d = p._asdict()                 # convert to a dictionary
    >>> d['x']
    11
    >>> Point(**d)                      # convert from a dictionary
    Point(x=11, y=22)
    >>> p._replace(x=100)               # _replace() is like str.replace() but targets named fields
    Point(x=100, y=22)



In [3]:
Person = namedtuple('Person', 'name age dob')

Person('A', 12, '1994-04-04')

Person(name='A', age=12, dob='1994-04-04')

In [5]:
field_names = tuple()
def __init__(self, *args, **kwargs):
    attrs = dict(zip(self.__slots__, args))
    attrs.update(kwargs)
    for name, value in attrs.items():
        setattr(self, name, value)
def __iter__(self):
    for name in self.__slots__:
        yield getattr(self, name)
def __repr__(self):
    values = ', '.join('{}={!r}'.format(*i) for i in zip(self.__slots__, self))
    return '{}({})'.format(self.__class__.__name__, values)

cls_attrs = dict(
    __slots__ = field_names,
    __init__ = __init__,
    __iter__ = __iter__,
    __repr__ = __repr__)

# how to make a class
cls_name='NEW_CLASS'
type(cls_name, (object, ), cls_attrs)

__main__.NEW_CLASS

In [7]:
'str'.__class__

str

In [8]:
str.__class__

type

In [12]:
# classes are objects, therefore each class must be an instance of some other class 🤯
type.__class__ is type(type)

True

SyntaxError: invalid character '🤯' (U+1F92F) (4168230873.py, line 1)

In [14]:
type(object), type(type) # object is an instance of type, type is a subclass of object, type is an instance of itself!! #wtf

(type, type)

In [19]:
# ONLY metaclasses are BOTH instances AND SUBClASSES of type
# all other classes are just isntances
import collections 

collections.abc.Iterable.__class__

abc.ABCMeta

In [20]:
# an exmample
print('<[100]> evalsupport module start')

def deco_alpha(cls):
    print('<[200]> deco_alpha')
    
    def inner_1(self):
        print('<[300]> deco_alpha:inner_1')
        
    cls.method_y = inner_1
    return cls

class MetaAleph(type):
    print('<[400]> MetaAleph body')

    # same argument inputs as type()
    def __init__(cls, name, bases, dic):
        print('<[500]> MetaAleph.__init__')
        
        def inner_2(self):
            print('<[600]> MetaAleph.__init__:inner_2')
            
        cls.method_z = inner_2
        
print('<[700]> evalsupport module end')

<[100]> evalsupport module start
<[400]> MetaAleph body
<[700]> evalsupport module end


In [21]:
class ClassFive(metaclass=MetaAleph):
    print('ClassFive body')

    def __init__(self, ):
        print('classfive init')

    def method_z(self, ):
        print('ClassFive.method_z')
    

ClassFive body
<[500]> MetaAleph.__init__


In [22]:
c = ClassFive()
c.method_z()

classfive init
<[600]> MetaAleph.__init__:inner_2


In [25]:
# prepare happens before new, which happens before init
from typing import Any, Mapping

class Meta(type):
    @classmethod
    def __prepare__(
        metacls,
        __name: str,
        __bases: tuple[type, ...], 
        **kwds: Any) -> Mapping[str, object]:

        # return super().__prepare__(__name, __bases, **kwds) # default
        return collections.OrderedDict()

    def __init__(cls, __name, __bases, __attr_dict):
        super().__init__(__name, __bases, __attr_dict)
        cls._field_names = []
        
        for key, attr in __attr_dict.items():
            if isinstance(attr, Validated):
                type_name = type(attr).__name__
                

# PEP 487

Replaced metaclass, used `__init_subclass__` hook in class body and ahook to initizlie attributes

In [6]:
help(object.__init_subclass__)

Help on built-in function __init_subclass__:

__init_subclass__(...) method of builtins.type instance
    This method is called when a class is subclassed.
    
    The default implementation does nothing. It may be
    overridden to extend subclasses.



In [23]:
class A:
    initialized = False
    def __init_subclass__(cls, **kwargs):
        # cls is the subclass here
        super().__init_subclass__(**kwargs) # this ensures all subclases down are initialized
        cls.initialized = True
        cls.kwargs = kwargs

class B(A, x=3):
    pass

a = A()
a.initialized


False

In [24]:
def print_info_for_cls(cls, super):
    print('    cls                   =', cls)
    print('    cls.__mro__           =', cls.__mro__)
    print('    cls.__bases__         =', cls.__bases__)
    print('    cls.__base__          =', cls.__base__)
    print('    super()               =', super)
    print('    super().__thisclass__ =', super.__thisclass__)


def print_info_for_self(self, super):
    print('    self                     =', self)
    print('    self.__class__           =', self.__class__)
    print('    self.__class__.__mro__   =', self.__class__.__mro__)
    print('    self.__class__.__bases__ =', self.__class__.__bases__)
    print('    self.__class__.__base__  =', self.__class__.__base__)
    print('    super()                  =', super)
    print('    super().__thisclass__    =', super.__thisclass__)
# --------------------------------------------------


In [49]:
class Base:
    def __init__(self, ):
        print('Base __init__')
        print_info_for_self(self, super())
        super().__init__()

    def __init_subclass__(cls, **kwargs):
        print('Base.__init_subclass__')
        print('kwargs', kwargs)
        print_info_for_cls(cls, super())
        super().__init_subclass__()

In [50]:
class MixinA:
    def __init__(self):
        print('< MixinA.__init__')
        print_info_for_self(self=self, super=super())
        super().__init__()
        print('> MixinA.__init__')

    @classmethod
    def __init_subclass__(cls):
        print('< MixinA.__init_subclass__')
        print_info_for_cls(cls=cls, super=super())
        super().__init_subclass__()
        print('> MixinA.__init_subclass_')


class MixinB:
    def __init__(self):
        print('< MixinB.__init__')
        print_info_for_self(self=self, super=super())
        super().__init__()
        print('> MixinB.__init__')

    @classmethod
    def __init_subclass__(cls):
        print('< MixinB.__init_subclass__')
        print_info_for_cls(cls=cls, super=super())
        super().__init_subclass__()
        print('> MixinB.__init_subclass__')


In [51]:
class MyClass(Base, MixinA, x=3): # any kwargs here are passed to the parent's __init_subclass__
    def __init__(self):
        print('< MyClass.__init__')
        print_info_for_self(self=self, super=super())
        super().__init__()
        print('> MyClass.__init__')


Base.__init_subclass__
kwargs {'x': 3}
    cls                   = <class '__main__.MyClass'>
    cls.__mro__           = (<class '__main__.MyClass'>, <class '__main__.Base'>, <class '__main__.MixinA'>, <class 'object'>)
    cls.__bases__         = (<class '__main__.Base'>, <class '__main__.MixinA'>)
    cls.__base__          = <class '__main__.Base'>
    super()               = <super: <class 'Base'>, <MyClass object>>
    super().__thisclass__ = <class '__main__.Base'>
< MixinA.__init_subclass__
    cls                   = <class '__main__.MyClass'>
    cls.__mro__           = (<class '__main__.MyClass'>, <class '__main__.Base'>, <class '__main__.MixinA'>, <class 'object'>)
    cls.__bases__         = (<class '__main__.Base'>, <class '__main__.MixinA'>)
    cls.__base__          = <class '__main__.Base'>
    super()               = <super: <class 'MixinA'>, <MyClass object>>
    super().__thisclass__ = <class '__main__.MixinA'>
> MixinA.__init_subclass_


In [81]:
class Meta(type):
    def __new__(cls, name, bases, class_dict):
        print((cls, name, bases, class_dict))

        return type.__new__(cls, name, bases, class_dict)



class MyClass(metaclass=Meta):
    pass

# this is output AT TIME OF CLASS CREATION, NOT INSTANCE CREATION, at this stage the metaclass's __new__ is called

(<class '__main__.Meta'>, 'MyClass', (), {'__module__': '__main__', '__qualname__': 'MyClass'})


In [79]:
import abc


class AbstractBase(metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def must_implement_this_method(self):
        raise NotImplementedError()


class ConcreteChild(AbstractBase):
    pass

d = ConcreteChild()

TypeError: Can't instantiate abstract class ConcreteChild with abstract method must_implement_this_method

In [88]:
from abc import abstractmethod

class BaseClass:
    def __init_subclass__(cls, **kwargs):
        print(getattr(cls().foo, '__isabstractmethod__', False))
        super().__init_subclass__(**kwargs)

    @abstractmethod
    def foo(self, ):
        raise NotImplementedError

class Class(BaseClass):
    def foo(self, x):
        return x**2

False


In [84]:
Base.__abstractmethods__

AttributeError: __abstractmethods__

# PEP 3115

In [1]:
# __prepaer__ invoked as a function before class body evaluation
# takes two arguments (cls, bases)

# Example Usages
[Pandas Library: to register holidays](https://github.com/pandas-dev/pandas/blob/2be9661853f4e425e00e3a32d265fe889b242f44/pandas/tseries/holiday.py)  
[cuDF overriding methods](https://github.com/rapidsai/cudf/blob/6ca2ceb8e200d55f1f681a4ca086614a28d67ad1/python/cudf/cudf/core/index.py)  
[cuML for fixing init](https://github.com/rapidsai/cuml/blob/50716cf98c4103aa8dbbcc4ea64897ccb7a70722/python/cuml/internals/base_helpers.py)

# Using `__init_subclass__` to enforce function implementation

This is going to get very weird, very quickly. When you create a class in Python, it looks about like the following:

In [89]:
class MyClass:
    pass

Now, let's say I create some really cool class, with a set of cool functions, but I expect my users to implement some of the functions:

In [90]:
from abc import abstractmethod

class BaseClass:
    @abstractmethod
    def foo(self,):
        raise NotImplementedError

So the intention is, when my user inherits the above class, they do the following:

In [91]:
class UserClass(BaseClass):
    def foo(self, *args, **kwargs):
        # actual functionality
        pass

That's all well and good, but what happens if my user *forgets* to implement `foo`? The above ran just fine, and even instantiation works!

In [98]:
class BaseClass:
    @abstractmethod
    def foo(self,):
        raise NotImplementedError

class UserClass(BaseClass):
    pass

user_instance = UserClass()

Now, this is a problem. Suppose this class were deployed to some production system, which attempts to call `foo`...

In [100]:
user_instance.foo()

NotImplementedError: 

That's a problem! Any code that will fail should fail *at compile time*, NOT only after it's deployed. So how do you ensure that, given you write a class, users of your class actually implement the function?

Enter PEP 487: this PEP proposed a hook (Python's runtime is quite rich, an a hook is a concrete method in an abstract class that can be overridden by subclasses) for easing the customization of class creation:

In [114]:
from dis import dis

class Base:
    def __init_subclass__(cls, **kwargs):
        print('__init_subclass__ run', cls)

        super().__init_subclass__(**kwargs)

class MyClass(Base):
    def __init__(self, ):
        return 

__init_subclass__ run <class '__main__.MyClass'>


From the above, we can see the `__init_subclass__` is run *at time of class creation*. This is going to be useful to check for whether or not a user overrides my abstract function.

So let's try this again, in the `__init_subclass__`, we check whether or not the method `foo` is still abstract or not. In this case, methods decorated with `@abstractmethod` have an attribute `__isabstractmethod__` which can be pulled:

In [121]:
class BaseClass: # this is the class I would write
    def __init_subclass__(cls, **kwargs):
        # if attribute foo of the class cls is still abstract, raise an error
        if getattr(cls().foo, '__isabstractmethod__', False): 
            raise NotImplementedError('Function foo must be implemented')

        super().__init_subclass__(**kwargs)

    @abstractmethod
    def foo(self, ):
        raise NotImplementedError

Now if the above was set up correctly, any classes inheriting from `BaseClass` should fail to be created at all at time of **class** creation, NOT instance creation!

In [129]:
class MyGoodUserClass(BaseClass):
    def foo(self, x):
        return x**2

user_instance = MyGoodUserClass()
user_instance.foo(x=3)

9

The above works fine, the method `foo` was successfully overridden and implemented; but the best-case scenario is fairly uninteresting. What happens when a user *forgets* to implement/override `foo`?

In [130]:
class MyBadUserClass(BaseClass):
    pass

NotImplementedError: Function foo must be implemented

That's right, **class** creation fails up-front, exactly where it's supposed to fail! 

## An Actual Example
Okay that was quite meta (pun intended), let's see an example; Let's say, I have a parent class that does data transformations, but I expect the user to implement their own cost function, so the function should take two inputs and return the similarity between them:

In [138]:
import math
from abc import abstractmethod

class TransformData:
    def __init_subclass__(cls, **kwargs):
        if getattr(cls().cost , '__isabstractmethod__', False):
            raise NotImplementedError('Implement cost function!')

        super().__init_subclass__(**kwargs)

    # assume some useful functions here
    def exponent(self, x):
        return math.exp(x) 

    def factorial(self, x):
        return math.factorial(x)
    
    @abstractmethod
    def cost(self, a, b):
        raise NotImplementedError


Now, my user, by means of subclassing `TransformData`, must implement their own cost function. If they don't:

In [140]:
class UserTransforms(TransformData):
    pass

NotImplementedError: Implement cost function!

And if they do:  

In [141]:
class UserTransforms(TransformData):
    def cost(self, a, b):
        return a - b 

It goes without saying, this is for sake of example, and not *every* abstract method need necessarily be implemented. This is for mission-critical functionality where the entire purpose of the class is negated without implementation. 